# Índice de áreas úmidas

## Importando pacotes e inicializando *geemap*

In [2]:
import os
import ee
import geemap
import geemap.colormaps as cm

In [3]:
geemap.ee_initialize()

O índice de áreas úmidas é uma combinação da densidade de áreas úmidas localmente com a densidade e quantidade de áreas úmidas regionalmente. O índice de áreas úmidas entra no cálculo da diversidade da paisagem em locais planos e úmidos, com baixa variedade de formas de relevo e baixa amplitude altitudinal. Nesses locais a variação microclimática seria baixa devido a baixa variabilidade topográfica e geomorfológica, mas como há alta densidade de áreas úmidas, esses locais atuam regulando a variabilidade microclimática localmente [@anderson_resilient_2016].

Existem três cenários de distribuição de áreas úmidas:

* (1) Os locais estão presentes em áreas com alta densidade de áreas úmidas no entorno,
* (2) Os locais estão situados em áreas com baixa densidade de áreas úmidas localmente, mas alta densidade regionalmente,
* (3) Os locais estão presentes em áreas com alta quantidade de áreas úmidas, mas baixa densidade devido a sua distribuição espacial. 

Dessa forma, o índice de áreas úmidas é composto pelos três cenários citados anteriormente, primeiro é calculado a densidade local, regional e a quantidade regional e, para cada métrica, é calculado um valor de Z, subtraindo pela média e dividindo pelo desvio padrão. A densidade de áreas úmidas é a média ponderada dos valores de Z da densidade local e regional (peso duplo para a densidade local). Nos locais onde os valores de Z da quantidade de áreas úmidas regional é maior que a densidade média calculada anteriormente, o índice torna-se a média ponderada da densidade local, densidade regional e quantidade de áreas úmidas regional (duplo peso para a densidade local) [@anderson_resilient_2016].   


$$
\tiny
índice\ de\ áreas\ úmidas = 
  \begin{cases}  
    \frac{2 \times densidade_{local}(Z) + densidade_{regional}(Z)}{3} & \text{se densidade média é maior ou igual à quantidade regional}\\
    \frac{2 \times densidade_{local}(Z) + densidade_{regional}(Z) + quantidade_{regional}(Z)}{4}& \text {se a quantidade regional for maior a densidade média}
  \end{cases}
$$

Neste capítulo demonstraremos como as densidades e a quantidade de áreas úmidas foram calculadas e no capítulo [Z_scores](https://resiliencia-climatica-brasil.github.io/diversity-resilience-python/5_Z_scores.html) mostraremos como calculamos os valores de Z e o índice de áreas úmidas. 

## Banco de dados

Utilizamos como base de dados de áreas úmidas o *Global Wetlands database* [@gumbricht_tropical_2017]. Reprojetamos o *raster* de áreas úmidas para a mesma resolução do modelo digital de elevação usado nas etapas anteriores (~ 92 metros).

Retiramos as áreas úmidas classificadas como sistemas lacustres e ribeirinhos (*riverines* e *lacustrines*) e reclassificamos o *raster* como sendo área úmida (1) ou não sendo área úmida (0). Depois, calculamos a densidade de áreas úmidas dentro de uma vizinhaça de 450 metros (5 células) de raio de um *kernel* circular (densidade local). Calculamos também a densidade e a quantidade de áreas úmidas na vizinhança de ~ 1170 metros (13 células) (regional).


## Código para calcular as densidades e quantidade de áreas úmidas

As análises foram rodadas no *Google Earth Engine* [@gorelick_google_2017], devido a demanda computacional do projeto, usando o pacote **geemap** [@wu_geemap_2020] em *Python* [@python_software_foundation_python_2023] como interface pela facilidade na documentação e reprodutividade das análises. O JupyterNotebook para reprodução das análises pode ser encontrado em [https://github.com/Resiliencia-climatica-Brasil/diversity-resilience-python/blob/master/jupyternotebook/3_wetland_score.ipynb](https://github.com/Resiliencia-climatica-Brasil/diversity-resilience-python/blob/master/jupyternotebook/3_wetland_score.ipynb).


Primeiro, nós importamos os polígonos dos biomas do Brasil e extraímos suas coordenadas geográficas máximas e mínimas para delimitar a região de análise. Importamos o modelo digital de elevação e o *raster* de áreas úmidas e reprojetamos a resolução das áreas úmidas para a resolução do modelo digital de elevação.  

In [3]:
# Importando mapa de biomas do IBGE para extrair as coordenadas 
# mínimas e máximas do Brasil
regiao = ee.FeatureCollection("projects/ee-lucasljardim9/assets/Biome")

def func_cmp(feature):
    return feature.bounds() 

# Extraindo as coordenadas mínimas e máximas do Brasil
regiao_box = regiao.map(func_cmp).geometry().dissolve(**{'maxError': 1}).bounds()

# Extraindo a resolução do mapa
DEM = ee.Image("MERIT/DEM/v1_0_3") 

escala = DEM.projection().nominalScale()

# Reprojetando áreas úmidas
wetlands = (ee.Image("projects/ee-lucasljardim9/assets/Cifor_wetlands") 
            .reproject(**{'crs': "EPSG:4326",
                       'scale': escala}))

Em seguida, retiramos os sistemas ribeirinhos e lacustres do *raster* de áreas úmidas e reclassificamos as classes do raster em presença e ausência de áreas úmidas. 

In [4]:
# Criando uma máscara para rios e lagos

rivers = wetlands.mask(wetlands.neq(10))

# Retirando rios e lagos das áreas úmidas

wetlands = wetlands.mask(rivers)

# Transformando áreas úmidas em um raster binário
# de presença de áreas úmidas

wetlands_binary = wetlands.where(wetlands.gt(0), 1).unmask()

Posteriomente, calculamos a densidade de áreas úmidas, localmente, dentro de um *kernel* circular de ~450 metros (5 células). Primeiro, transformamos as células da vizinhança de cada célula focal em bandas de uma imagem. Assim, cada células vizinha da célula focal fica empilhada como uma coluna. Para cada coluna, somamos os valores das células (0 ou 1) como a quantidade de áreas úmidas na vizinhança. Depois, dividimos a quantidade de áreas úmidas pelo número de células na vizinhança, resultando na densidade de áreas úmidas.

In [5]:
radius_pixels = 5

# Tranforme as células da vizinha em bandas

neighbors = (wetlands_binary
            .neighborhoodToBands(ee.Kernel.circle(ee.Number(radius_pixels))))

# Conte a quantidade de áreas úmidas na vizinhança
wetlands_count = neighbors.reduce(ee.Reducer.sum()).toDouble()

# Conte o número de células totais na vizinhança

neighbors_amount = neighbors.bandNames().length()

#Divida a quantidade de áreas úmidas pelo 
# número de células para calcular a densidade 
wetlands_density_local = wetlands_count.divide(ee.Number(neighbors_amount))

Repetimos o mesmo procedimento conduzido para a densidade local para calcularmos a densidade e quantidade de áreas úmidas regional.

In [6]:
radius_pixels = 13

# Transforme as células vizinhas em bandas

neighbors = (wetlands_binary
           .neighborhoodToBands(ee.Kernel.circle(
                                   ee.Number(radius_pixels)
                                   )
                               ))

# Conte a quantidade de áreas úmidas
wetlands_count = neighbors.reduce(ee.Reducer.sum()).toDouble()

# Conte o número de células totais na vizinhança
neighbors_amount = neighbors.bandNames().length()

# Calcule a densidade dividindo a quantidade 
# de áreas úmidas pelo número de células
wetlands_density_regional = wetlands_count.divide(ee.Number(neighbors_amount))

Por fim, exportamos a densidade de áreas úmidas local (*wetlands_density*), a densidade regional (*wetlands_density_1000*) e a quantidade regional (*wetlands_count*) como *assets* no *Google Earth Engine*.

In [7]:
# modifique os assetIds para os do seu projeto

assetId_quantidade = "projects/ee-lucasljardim9/assets/wetlands_count"

assetId_densidade_local = "projects/ee-lucasljardim9/assets/wetlands_density"

assetId_densidade_regional = "projects/ee-lucasljardim9/assets/wetlands_density_1000"

geemap.ee_export_image_to_asset(
    wetlands_count, 
    description='wetlands_count', 
    assetId=assetId_quantidade, 
    region=regiao_box, 
    scale=escala, maxPixels=1e13
)

geemap.ee_export_image_to_asset(
    wetlands_density_local, 
    description='wetlands_density_local', 
    assetId=assetId_densidade_local, 
    region=regiao_box, 
    scale=escala, maxPixels=1e13
)

geemap.ee_export_image_to_asset(
    wetlands_density_regional, 
    description='wetlands_density_regional', 
    assetId=assetId_densidade_regional, 
    region=regiao_box, 
    scale=escala, maxPixels=1e13
)

Os dados criados podem ser visualizados como abaixo:

In [9]:
# Paleta de cores

palette = cm.get_palette('terrain', n_class=10)

# Centralize o mapa

Map = geemap.Map(center=(-11.75, -51.52), zoom=4)

# Importe os assets

wetlands_count = ee.Image("projects/ee-lucasljardim9/assets/wetlands_count")

wetlands_density_local = ee.Image("projects/ee-lucasljardim9/assets/wetlands_density")

wetlands_density_regional = ee.Image("projects/ee-lucasljardim9/assets/wetlands_density_1000")


# Plote o mapa

Map.addLayer(wetlands_count, {'palette':palette, 'min':0, 'max': 1}, "wetlands count") 
Map.addLayer(wetlands_density_local, {'palette':palette, 'min':0, 'max': 1}, "local density") 
Map.addLayer(wetlands_density_regional, {'palette':palette, 'min':0, 'max': 1}, "regional density") 



Map

Map(center=[-11.75, -51.52], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…